## "Oro Verde" Carbon Offset Project
This notebook implements a compact solution to Q3 for the Oro Verde mini-case. The notebook assumes that you are  familiar with the Oro Verde case and have understood the basic optimization model.

### Import any relevant Python packages

In [1]:
# import the entire Gurobi package (used for linear optimization)
from gurobipy import *

# import the pandas package for reading data files
import pandas as pd

### Read & store the problem parameters / data

In [2]:
land_avail = 150000              # available land area (in square feet)
water_avail = 50000              # available water (in gallons)
annual_seq_commit = 1800              # annual sequestration commitment
min_elms = 15   # minimum requirement on elm trees

# Use the _pandas_ module to create a data frame called "mydata" by reading the sheet called "Data" from an Excel file
mydata = pd.read_excel("S01_ARM_Oro_Verde_data.xlsx", sheet_name = "Data", index_col = 0) 

# print out the data frame to have a quick look
mydata

,SequestrationRate,WaterRequirement,Width,SurvivalRate,SeedlingCost,SeedlingsAvailable
TreeType,,,,,,
Maple,4.3,76,130,0.7,5.4,1000
Elm,3.7,48,3600,0.6,3.2,500
Spruce,2.8,34,400,0.8,4.5,1500


In [3]:
# create a list with all the types of trees (this will be very useful for creating our decision variables)
tree_types = list(mydata.index)

print(tree_types)

['Maple', 'Elm', 'Spruce']


# Set up the Optimization Model
### Create empty Gurobi model

In [4]:
mymodel = Model("Oro Verde Tree Planting Model")

Using license file /Users/daniancu/gurobi.lic
Academic license - for non-commercial use only


### Create and add decision variables into the model

In [5]:
trees_to_plant = mymodel.addVars(tree_types)

### Formulate and add the objective to the model

In [6]:
# set the objective in the Gurobi model
mymodel.setObjective( quicksum( trees_to_plant[t] * mydata["SeedlingCost"][t] for t in tree_types), GRB.MINIMIZE)

### Formulate and add the constraints

In [7]:
# water availability
mymodel.addConstr( quicksum( trees_to_plant[t] * mydata["WaterRequirement"][t] for t in tree_types) <= water_avail )

# land availability
mymodel.addConstr( quicksum( trees_to_plant[t] * mydata["Width"][t] for t in tree_types) <= land_avail )

# carbon sequestration commitment
mymodel.addConstr( quicksum( trees_to_plant[t] * mydata["SurvivalRate"][t] * mydata["SequestrationRate"][t] \
             for t in tree_types ) >= annual_seq_commit )

# minimum elm trees
mymodel.addConstr( trees_to_plant["Elm"] >= min_elms )

# seedling availability for each tree type
for t in tree_types :
    mymodel.addConstr( trees_to_plant[t] <= mydata["SeedlingsAvailable"][t] )

### Solve/optimize the model

In [8]:
mymodel.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 7 rows, 3 columns and 13 nonzeros
Model fingerprint: 0xd6e51bd9
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+05]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1965890e+03   1.154615e+03   0.000000e+00      0s
       1    3.2130952e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.213095170e+03


# Extract the optimized results

In [9]:
# print the value of the objective
print("Optimal cost is: %.3f"  % mymodel.objVal)

Optimal cost is: 3213.095


In [10]:
# print solution with a for loop
print("Optimal solution is to plant:")
for t in tree_types :
    print( "%10s : %5.3f" % ( t, trees_to_plant[t].X ) )

Optimal solution is to plant:
     Maple : 582.798
       Elm : 20.621
    Spruce : 0.000
